In [78]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark-2.2.0-bin-hadoop2.7/")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
import pyspark.mllib.linalg.distributed
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [79]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [80]:
file_name = "/Users/simondi/PHD/data/data/target_infect_x/query_data/cells_sample_10_normalized.tsv"
df = spark.read.csv(path=file_name, sep="\t", header='true')

In [81]:
old_cols = df.columns
new_cols = list(map(lambda x: x.replace(".", "_"), old_cols))

df = reduce(
  lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]),
  range(len(new_cols)), df)

for i, x in enumerate(new_cols):
    if x.startswith("cells") or x.startswith("perin") or x.startswith("nucl"):
        df = df.withColumn(x, df[x].cast("double"))

df = df.fillna(0)

In [82]:
feature_columns = [x for x in df.columns if x.startswith("cells") or x.startswith("perin") or x.startswith("nucl")]
assembler = VectorAssembler(inputCols=feature_columns,outputCol='features')
data = assembler.transform(df)

In [83]:
rdd = data.select(feature_columns).rdd
X = RowMatrix(rdd)

In [87]:
spark.version

'2.1.1'

In [84]:
def svd(X, n_components):
    svd = X.computeSVD(n_components, computeU=True)
    U = svd.U       # The U factor is a RowMatrix.
    s = svd.s       # The singular values are stored in a local dense vector.
    V = svd.V       # The V factor is a local dense matrix.
    return U, s, V

In [85]:
U, s, V = svd(X, 2)

AttributeError: 'RowMatrix' object has no attribute 'computeSVD'

In [77]:
spark.stop()